In [1]:
%matplotlib inline

import sys
sys.path.append('/Users/paepcke/anaconda2/envs/women_tech/lib/python2.7/site-packages')
sys.path

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
import numpy as np
import pandas as pn
import os
import survey_utils
import xlrd

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation

/Users/paepcke/anaconda2/envs/women_tech/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_path = os.path.join(os.getenv("HOME"), 
                        'Project/WomenIndustrySurveyHolzblatt/Data/DataForClustering/likertSlidersByRespondentUnfolded.xlsx')
#out_path = os.path.splitext(data_path)[0] + 'Unfolded.csv'
#utils.unfold(data_path, out_path)
#dataPath = out_path

In [3]:
data = pn.read_excel(data_path)
data

,question,s0,s1,s2,s3,s4,s5,s6,s7,s8,...,s208,s209,s210,s211,s212,s213,s214,s215,s216,s217
0,PersonalExperience|ILackConfidence,35,82,35,53,35,82,47,100,59,...,0,0,0,0,0,0,0,0,0,0
1,WorkLife|ReceivedBadReview,59,94,94,65,94,82,94,100,94,...,0,0,0,0,0,0,0,0,0,0
2,Experience|Bored,12,12,12,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
3,Feel|ValuedByManager,12,12,12,12,12,12,12,12,12,...,87,87,87,87,87,87,87,87,87,87
4,Power|HaveSelfConfidence,12,12,12,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
5,Team|GetsInWay,76,88,65,71,88,47,94,100,53,...,0,0,0,0,0,0,0,0,0,0
6,Team|HoHum,82,53,88,53,65,65,88,100,53,...,0,0,0,0,0,0,0,0,0,0
7,Power|MenGetGoodReactions,12,12,12,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
8,Promoted|DontWantClimb,12,12,12,12,12,12,12,12,12,...,0,0,0,0,0,0,0,0,0,0
9,Team|PowerStruggles,76,47,59,29,94,24,82,24,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Make the question column the index, so that
# all operations on the dataframe only address
# the numeric scores:
data = data.set_index('question')
# Replace 0s with their column's median.
#data = utils.replaceZerosDataFrame(data, direction='row')
row1 = data.iloc[1]
#print('Len of row: %s' % len(row1))
#print('row.value_counts(): %s' % row1.value_counts())
row1.value_counts()[0]
ninetyPercOfLen = len(row1)*90/100
#print('90perc of len: %s' % ninetyPercOfLen)
#***data.ix[:,'s0':'s212']
percZeroes = 100*row1.value_counts()[0]/len(row1)
(row1==0).sum()
#print('Len data: %s' % len(data))
cols_to_exclude = ['question', 'questionType', 'outputVar']
for row in data.itertuples():
    # Get array of just the measurements (no 'question', questionType', 'outputVar'):
    row_values = pandas.Series(row[3:])
    # Kludge: get array of True/False; these
    # are coded as 1/0:
    num_zeros      = (row_values==0).sum()
    perc_zeros     = 100 * num_zeros / len(row_values)
    perc_completed = 100-perc_zeros
    #***print(len(row_values), num_zeros, perc_zeros, perc_completed)
    #********
    curr_row = 0
    if (row_values.ix[100:217] !=0).any():
        #print('Row %s: %s' % (curr_row, row_values.ix[200:]))
        #print('Non-zero Likert: %s' % row_values)
        curr_row += 1
    #********
data    



Create info on successively clustering data
into ever larger clusters. Use e.g. the Ward
option, which measures distance between data items
by the difference in their variance (the more similar
the variance the closer they are):
           Feature1   Feature2      Distance   SizeResultingCluster
   array([[  52.     ,   53.     ,    0.04151,    2.     ],
          [  14.     ,   79.     ,    0.05914,    2.     ],
          [  33.     ,   68.     ,    0.07107,    2.     ],
          [  17.     ,   73.     ,    0.07137,    2.     ],
          [   1.     ,    8.     ,    0.07543,    2.     ],
                       ...
          [  62.     ,  152.     ,    0.1726 ,    3.     ],
          [  41.     ,  158.     ,    0.1779 ,    3.     ],
                       ...
          
First line: feature 52 is combined with feature 53 into
a cluster of size 2. Distance in variance between them 
is 0.04151. First cluster of 3: item 62 with 152. The dataset
of this table had data items 0-149. Any 'data item' with a 
number 150 or higher refers to an already formed cluster.
To find items that made up this cluster: 
     theMergeLine = lineGTDataLength - datalength

In [ ]:
#data

In [ ]:
#clusterInfo = linkage(data, 'ward')      # c=0.62
#clusterInfo = linkage(data, 'centroid')  # c=0.89
#clusterInfo = linkage(data, 'weighted')  # c=0.86
#clusterInfo = linkage(data, 'average')   # c=0.91
#clusterInfo = linkage(data, 'complete')  # c=0.90
#clusterInfo = linkage(data, 'single')    # c=0.78

# Cophenet correlation coefficient measures 
# how faithfully a dendrogram preserves pairwise
# distance between the original data points:
(c, coph_dists) = cophenet(clusterInfo, pdist(data))
c

In [ ]:
pandas.DataFrame(clusterInfo[:20],columns=['feature1', 'feature2', 'distance', 'clusterSize'])

In [ ]:
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Survey Question')
plt.ylabel('Distance')
dendrogram(
    clusterInfo,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=16.,  # font size for the x axis labels
    labels=data.index,  # question names
)
plt.show()



In [ ]:
# set cut-off to 50
max_d = 750 # max_d as in max_distance

In [ ]:
plt.figure(figsize=(25, 10))
utils.fancy_dendrogram(
    clusterInfo,
    x_label='Survey Question',
    #***truncate_mode='lastp',
    p=4,
    leaf_rotation=90.,
    leaf_font_size=10.,
    x_axis_font_size=16.,
    y_axis_font_size=16.,
    labels=dataNew.index,
    show_contracted=True,
    annotate_above=10000, # Make high enough never to get distance labels in chart
    max_d=max_d,  # plot a horizontal cut-off line
)
plt.show()

